In [ ]:
# 출처 : https://wikidocs.net/233344
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [ ]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

df = pd.read_csv("./data/titanic/train.csv")
# titanic/train.csv 파일에서 데이터를 읽어와 DataFrame으로 저장합니다.
tool = PythonAstREPLTool(locals={"df": df})

# PythonAstREPLTool을 사용하여 로컬 변수 'df'를 포함하는 환경을 생성합니다.
tool.invoke("df")
# 'df' DataFrame에서 'Fare' 열의 평균 값을 계산합니다.

In [12]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

# PythonAstREPLTool을 이용하여 수치적으로 계산하는 예

prompt = """
print("Hello, world!")

def add(a, b):
    return a + b

print(add(30, 40))

import pandas as pd

df = pd.read_csv("./data/titanic/train.csv")
df.head()
"""

tool = PythonAstREPLTool(locals={"df": df})
# PythonAstREPLTool을 사용하여 로컬 변수 'df'를 포함하는 환경을 생성합니다.
tool.invoke(prompt)

Hello, world!
70


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# 데이터 로드

import pandas as pd
df = pd.read_csv("./data/titanic/train.csv")

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.callbacks.base import BaseCallbackHandler


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


# 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo-0613",
        streaming=True,
        callbacks=[StreamCallback()],
    ),  # 모델 정의
    df,  # 데이터프레임
    verbose=True,  # 추론과정 출력
    # AgentType.ZERO_SHOT_REACT_DESCRIPTION
    agent_type=AgentType.OPENAI_FUNCTIONS,  # 이걸해야함.(PythonAstREPLTool 함수와 같은 역활)
)


In [7]:
# 질의
agent.invoke({"input": "데이터의 행과 열의 갯수는 어떻게 돼?"})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape'}`


(891, 12)데이터프레임 `df`는 891개의 행과 12개의 열로 구성되어 있습니다.데이터프레임 `df`는 891개의 행과 12개의 열로 구성되어 있습니다.

> Finished chain.


{'input': '데이터의 행과 열의 갯수는 어떻게 돼?',
 'output': '데이터프레임 `df`는 891개의 행과 12개의 열로 구성되어 있습니다.'}

In [8]:
# 질의
agent.run("남자 승객의 생존율을 어떻게 돼? %로 알려줘")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "survival_rate = df[df['Sex'] == 'male']['Survived'].mean() * 100\nsurvival_rate"}`


18.890814558058924남자 승객의 생존율은 약 18.89%입니다.남자 승객의 생존율은 약 18.89%입니다.

> Finished chain.


'남자 승객의 생존율은 약 18.89%입니다.'

In [9]:
# df가 2개인 경우

# 샘플 데이터프레임 생성
df1 = df.copy()
df1 = df1.fillna(0)
df1.head()

# 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo-0613",
        streaming=True,
        callbacks=[StreamCallback()],
    ),  # 모델 정의
    [df,df1],  # 데이터프레임
    verbose=True,  # 추론과정 출력
    # AgentType.ZERO_SHOT_REACT_DESCRIPTION
    agent_type=AgentType.OPENAI_FUNCTIONS,  # 이걸해야함.
)

In [10]:
# df, df1 2개이므로, 2개를 비교하는 질문을 해야 함.
# => 1개 df와 연관된 질문을 하면 안됨.
agent.invoke({"input": "나이 컬럼의 나이의 평균차이는 어떻게 돼? %로 구해줘."})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df1['Age'].mean() - df2['Age'].mean()"}`


5.899824717765892나이 컬럼의 나이의 평균 차이는 약 5.9%입니다.나이 컬럼의 나이의 평균 차이는 약 5.9%입니다.

> Finished chain.


{'input': '나이 컬럼의 나이의 평균차이는 어떻게 돼? %로 구해줘.',
 'output': '나이 컬럼의 나이의 평균 차이는 약 5.9%입니다.'}